In [1]:
import mxnet as mx 
from Symbol.symbol import get_resnet_model_YoloV1
import numpy as np
from data_ulti import get_iterator
from tools.logging_metric import LogMetricsCallback, LossMetric
import time

import logging
import sys
root_logger = logging.getLogger()
stdout_handler = logging.StreamHandler(sys.stdout)
root_logger.addHandler(stdout_handler)
root_logger.setLevel(logging.DEBUG)

F:\Programsl\Anaconda3\lib\site-packages\urllib3\contrib\pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
# get sym
# Try different network 18, 50, 101 to find the best one
sym = get_resnet_model_YoloV1('pretrained_models/resnet-34', 0)
_, args_params, aux_params = mx.model.load_checkpoint('pretrained_models/resnet-34', 0)

# get some input
# change it to the data rec you create, and modify the batch_size
train_data = get_iterator(path='DATA_rec/drive_small.rec', data_shape=(3, 224, 224), label_width=7*7*9, batch_size=2, shuffle=True)
val_data = get_iterator(path='DATA_rec/drive_small.rec', data_shape=(3, 224, 224), label_width=7*7*9, batch_size=2)

In [3]:
# allocate gpu/cpu mem to the sym
mod = mx.mod.Module(symbol=sym, context=mx.cpu(0))
# save model
checkpoint = mx.callback.do_checkpoint('drive_small_detect')

In [4]:
# setup metric
# metric = mx.metric.create(loss_metric, allow_extra_outputs=True)
tme = time.time()
logtrain = LogMetricsCallback('logs/train_'+str(tme))

# setup monitor for debugging
def norm_stat(d):
    return mx.nd.norm(d) / np.sqrt(d.size)
mon = None #mx.mon.Monitor(10, norm_stat, pattern=".*backward*.")

# Train
# Try different hyperparamters to get the model converged, (batch_size,
# optimization method, training epoch, learning rate/scheduler)
mod.fit(train_data=train_data,
        eval_data=val_data,
        num_epoch=100,
        monitor=mon,
        eval_metric=LossMetric(0.5),
        optimizer='rmsprop',
        optimizer_params={'learning_rate':0.001, 'lr_scheduler': mx.lr_scheduler.FactorScheduler(300000, 0.1, 0.001)},
        initializer=mx.init.Xavier(magnitude=2, rnd_type='gaussian', factor_type='in'),
        arg_params=args_params,
        aux_params=aux_params,
        allow_missing=True,
        batch_end_callback=[mx.callback.Speedometer(batch_size=2, frequent=10, auto_reset=False), logtrain],
        epoch_end_callback=checkpoint,
         )

F:\_study 2018_01_06\2 bittiger-AI\bittiger course\bittigerProject\Capstone\objectDectection\tools\logging_metric.py:84: RuntimeWarning: invalid value encountered in double_scalars
  self.sum_x = np.sum((np.abs(pred[:, :, 1] - label[:, :, 1])) * c_label) * self.stride / num_boxes
F:\_study 2018_01_06\2 bittiger-AI\bittiger course\bittigerProject\Capstone\objectDectection\tools\logging_metric.py:85: RuntimeWarning: invalid value encountered in double_scalars
  self.sum_y = np.sum((np.abs(pred[:, :, 2] - label[:, :, 2])) * c_label) * self.stride / num_boxes
F:\_study 2018_01_06\2 bittiger-AI\bittiger course\bittigerProject\Capstone\objectDectection\tools\logging_metric.py:87: RuntimeWarning: invalid value encountered in double_scalars
  * self.W / num_boxes
F:\_study 2018_01_06\2 bittiger-AI\bittiger course\bittigerProject\Capstone\objectDectection\tools\logging_metric.py:89: RuntimeWarning: invalid value encountered in double_scalars
  * self.H / num_boxes


Epoch[0] Train-c_accuracy=1.000000
Epoch[0] Train-c_precision=0.000000
Epoch[0] Train-c_recall=0.000000
Epoch[0] Train-c_diff=0.074758
Epoch[0] Train-x_diff=nan
Epoch[0] Train-y_diff=nan
Epoch[0] Train-w_diff=nan
Epoch[0] Train-h_diff=nan
Epoch[0] Train-loss=0.014952
Epoch[0] Train-cls_diff=0.000000
Epoch[0] Time cost=18.636
Saved checkpoint to "drive_small_detect-0001.params"
Epoch[0] Validation-c_accuracy=0.989796
Epoch[0] Validation-c_precision=0.000000
Epoch[0] Validation-c_recall=0.000000
Epoch[0] Validation-c_diff=0.084641
Epoch[0] Validation-x_diff=1.905093
Epoch[0] Validation-y_diff=2.530165
Epoch[0] Validation-w_diff=5.244386
Epoch[0] Validation-h_diff=180.772242
Epoch[0] Validation-loss=0.146268
Epoch[0] Validation-cls_diff=0.001964
Epoch[1] Train-c_accuracy=0.979592
Epoch[1] Train-c_precision=0.000000
Epoch[1] Train-c_recall=0.000000
Epoch[1] Train-c_diff=0.101172
Epoch[1] Train-x_diff=11.916772
Epoch[1] Train-y_diff=1.831702
Epoch[1] Train-w_diff=17.247464
Epoch[1] Train-h_

Epoch[10] Validation-x_diff=6.549078
Epoch[10] Validation-y_diff=0.561167
Epoch[10] Validation-w_diff=4.202413
Epoch[10] Validation-h_diff=9.888909
Epoch[10] Validation-loss=0.103936
Epoch[10] Validation-cls_diff=0.000228
Epoch[11] Train-c_accuracy=0.918367
Epoch[11] Train-c_precision=0.111111
Epoch[11] Train-c_recall=0.999999
Epoch[11] Train-c_diff=0.128364
Epoch[11] Train-x_diff=2.537741
Epoch[11] Train-y_diff=9.541237
Epoch[11] Train-w_diff=11.117835
Epoch[11] Train-h_diff=1.622130
Epoch[11] Train-loss=0.166110
Epoch[11] Train-cls_diff=0.001618
Epoch[11] Time cost=13.548
Saved checkpoint to "drive_small_detect-0012.params"
Epoch[11] Validation-c_accuracy=0.938776
Epoch[11] Validation-c_precision=0.142857
Epoch[11] Validation-c_recall=0.999999
Epoch[11] Validation-c_diff=0.090212
Epoch[11] Validation-x_diff=6.801735
Epoch[11] Validation-y_diff=0.814840
Epoch[11] Validation-w_diff=6.343163
Epoch[11] Validation-h_diff=10.946016
Epoch[11] Validation-loss=0.098985
Epoch[11] Validation-cl

Epoch[21] Train-c_diff=0.138989
Epoch[21] Train-x_diff=5.728932
Epoch[21] Train-y_diff=0.356867
Epoch[21] Train-w_diff=0.057304
Epoch[21] Train-h_diff=0.529509
Epoch[21] Train-loss=0.121170
Epoch[21] Train-cls_diff=0.000488
Epoch[21] Time cost=13.790
Saved checkpoint to "drive_small_detect-0022.params"
Epoch[21] Validation-c_accuracy=0.989796
Epoch[21] Validation-c_precision=0.500000
Epoch[21] Validation-c_recall=0.999999
Epoch[21] Validation-c_diff=0.041221
Epoch[21] Validation-x_diff=6.599001
Epoch[21] Validation-y_diff=0.742193
Epoch[21] Validation-w_diff=6.002934
Epoch[21] Validation-h_diff=9.127376
Epoch[21] Validation-loss=0.091855
Epoch[21] Validation-cls_diff=0.000128
Epoch[22] Train-c_accuracy=0.959184
Epoch[22] Train-c_precision=0.200000
Epoch[22] Train-c_recall=0.999999
Epoch[22] Train-c_diff=0.082050
Epoch[22] Train-x_diff=5.413939
Epoch[22] Train-y_diff=0.412966
Epoch[22] Train-w_diff=0.498690
Epoch[22] Train-h_diff=0.128532
Epoch[22] Train-loss=0.110312
Epoch[22] Train-cl

Epoch[31] Validation-c_precision=0.250000
Epoch[31] Validation-c_recall=0.999999
Epoch[31] Validation-c_diff=0.054750
Epoch[31] Validation-x_diff=7.058258
Epoch[31] Validation-y_diff=0.827627
Epoch[31] Validation-w_diff=6.604337
Epoch[31] Validation-h_diff=8.961658
Epoch[31] Validation-loss=0.094731
Epoch[31] Validation-cls_diff=0.000148
Epoch[32] Train-c_accuracy=0.928571
Epoch[32] Train-c_precision=0.222222
Epoch[32] Train-c_recall=1.000000
Epoch[32] Train-c_diff=0.092294
Epoch[32] Train-x_diff=2.623869
Epoch[32] Train-y_diff=1.168383
Epoch[32] Train-w_diff=10.226827
Epoch[32] Train-h_diff=16.765946
Epoch[32] Train-loss=0.212349
Epoch[32] Train-cls_diff=0.002120
Epoch[32] Time cost=13.452
Saved checkpoint to "drive_small_detect-0033.params"
Epoch[32] Validation-c_accuracy=1.000000
Epoch[32] Validation-c_precision=0.999999
Epoch[32] Validation-c_recall=0.999999
Epoch[32] Validation-c_diff=0.047802
Epoch[32] Validation-x_diff=6.950755
Epoch[32] Validation-y_diff=0.674227
Epoch[32] Vali

...

Epoch[52] Validation-c_accuracy=0.979592
Epoch[52] Validation-c_precision=0.333333
Epoch[52] Validation-c_recall=0.999999
Epoch[52] Validation-c_diff=0.061217
Epoch[52] Validation-x_diff=6.228125
Epoch[52] Validation-y_diff=0.698299
Epoch[52] Validation-w_diff=6.212672
Epoch[52] Validation-h_diff=8.780425
Epoch[52] Validation-loss=0.097906
Epoch[52] Validation-cls_diff=0.000203
Epoch[53] Train-c_accuracy=0.979592
Epoch[53] Train-c_precision=0.600000
Epoch[53] Train-c_recall=1.000000
Epoch[53] Train-c_diff=0.070934
Epoch[53] Train-x_diff=1.489627
Epoch[53] Train-y_diff=3.164207
Epoch[53] Train-w_diff=3.591648
Epoch[53] Train-h_diff=2.719998
Epoch[53] Train-loss=0.328113
Epoch[53] Train-cls_diff=0.002907
Epoch[53] Time cost=13.538
Saved checkpoint to "drive_small_detect-0054.params"
Epoch[53] Validation-c_accuracy=0.979592
Epoch[53] Validation-c_precision=0.333333
Epoch[53] Validation-c_recall=0.999999
Epoch[53] Validation-c_diff=0.055842
Epoch[53] Validation-x_diff=5.755804
Epoch[53] Va

Epoch[62] Validation-cls_diff=0.000249
Epoch[63] Train-c_accuracy=0.979592
Epoch[63] Train-c_precision=0.333333
Epoch[63] Train-c_recall=0.999999
Epoch[63] Train-c_diff=0.049759
Epoch[63] Train-x_diff=2.417420
Epoch[63] Train-y_diff=0.425664
Epoch[63] Train-w_diff=1.471185
Epoch[63] Train-h_diff=1.521747
Epoch[63] Train-loss=0.085137
Epoch[63] Train-cls_diff=0.000950
Epoch[63] Time cost=15.413
Saved checkpoint to "drive_small_detect-0064.params"
Epoch[63] Validation-c_accuracy=1.000000
Epoch[63] Validation-c_precision=0.999999
Epoch[63] Validation-c_recall=0.999999
Epoch[63] Validation-c_diff=0.031719
Epoch[63] Validation-x_diff=5.295319
Epoch[63] Validation-y_diff=0.584124
Epoch[63] Validation-w_diff=5.459017
Epoch[63] Validation-h_diff=8.034474
Epoch[63] Validation-loss=0.094546
Epoch[63] Validation-cls_diff=0.000254
Epoch[64] Train-c_accuracy=1.000000
Epoch[64] Train-c_precision=0.000000
Epoch[64] Train-c_recall=0.000000
Epoch[64] Train-c_diff=0.034780
Epoch[64] Train-x_diff=nan
Epo

Epoch[73] Train-loss=0.221204
Epoch[73] Train-cls_diff=0.001968
Epoch[73] Time cost=14.658
Saved checkpoint to "drive_small_detect-0074.params"
Epoch[73] Validation-c_accuracy=1.000000
Epoch[73] Validation-c_precision=0.999999
Epoch[73] Validation-c_recall=0.999999
Epoch[73] Validation-c_diff=0.029838
Epoch[73] Validation-x_diff=5.809078
Epoch[73] Validation-y_diff=0.635386
Epoch[73] Validation-w_diff=5.710925
Epoch[73] Validation-h_diff=7.656928
Epoch[73] Validation-loss=0.092779
Epoch[73] Validation-cls_diff=0.000217
Epoch[74] Train-c_accuracy=0.989796
Epoch[74] Train-c_precision=0.000000
Epoch[74] Train-c_recall=0.000000
Epoch[74] Train-c_diff=0.033289
Epoch[74] Train-x_diff=nan
Epoch[74] Train-y_diff=nan
Epoch[74] Train-w_diff=nan
Epoch[74] Train-h_diff=nan
Epoch[74] Train-loss=0.006658
Epoch[74] Train-cls_diff=0.000000
Epoch[74] Time cost=14.087
Saved checkpoint to "drive_small_detect-0075.params"
Epoch[74] Validation-c_accuracy=1.000000
Epoch[74] Validation-c_precision=0.999999
E

Epoch[83] Validation-h_diff=8.096291
Epoch[83] Validation-loss=0.100474
Epoch[83] Validation-cls_diff=0.000205
Epoch[84] Train-c_accuracy=0.989796
Epoch[84] Train-c_precision=0.500000
Epoch[84] Train-c_recall=0.999999
Epoch[84] Train-c_diff=0.040861
Epoch[84] Train-x_diff=2.338914
Epoch[84] Train-y_diff=0.182232
Epoch[84] Train-w_diff=1.334587
Epoch[84] Train-h_diff=2.823720
Epoch[84] Train-loss=0.081622
Epoch[84] Train-cls_diff=0.000776
Epoch[84] Time cost=13.582
Saved checkpoint to "drive_small_detect-0085.params"
Epoch[84] Validation-c_accuracy=1.000000
Epoch[84] Validation-c_precision=0.999999
Epoch[84] Validation-c_recall=0.999999
Epoch[84] Validation-c_diff=0.024608
Epoch[84] Validation-x_diff=4.574854
Epoch[84] Validation-y_diff=0.749418
Epoch[84] Validation-w_diff=6.325786
Epoch[84] Validation-h_diff=9.599276
Epoch[84] Validation-loss=0.091321
Epoch[84] Validation-cls_diff=0.000165
Epoch[85] Train-c_accuracy=0.959184
Epoch[85] Train-c_precision=0.000000
Epoch[85] Train-c_recall

Epoch[94] Train-loss=0.110917
Epoch[94] Train-cls_diff=0.000408
Epoch[94] Time cost=13.436
Saved checkpoint to "drive_small_detect-0095.params"
Epoch[94] Validation-c_accuracy=0.979592
Epoch[94] Validation-c_precision=0.333333
Epoch[94] Validation-c_recall=0.999999
Epoch[94] Validation-c_diff=0.059121
Epoch[94] Validation-x_diff=3.877367
Epoch[94] Validation-y_diff=0.837145
Epoch[94] Validation-w_diff=6.672736
Epoch[94] Validation-h_diff=9.253988
Epoch[94] Validation-loss=0.112605
Epoch[94] Validation-cls_diff=0.000115
Epoch[95] Train-c_accuracy=0.989796
Epoch[95] Train-c_precision=0.750000
Epoch[95] Train-c_recall=1.000000
Epoch[95] Train-c_diff=0.060599
Epoch[95] Train-x_diff=5.398163
Epoch[95] Train-y_diff=1.921683
Epoch[95] Train-w_diff=4.012185
Epoch[95] Train-h_diff=6.431342
Epoch[95] Train-loss=0.302743
Epoch[95] Train-cls_diff=0.002838
Epoch[95] Time cost=13.561
Saved checkpoint to "drive_small_detect-0096.params"
Epoch[95] Validation-c_accuracy=0.979592
Epoch[95] Validation-c_

In [ ]:
# # no tensorboard vision#################
# # Train
# # Try different hyperparamters to get the model converged, (batch_size,
# # optimization method, training epoch, learning rate/scheduler)
# mod.fit(train_data=train_data,
#         eval_data=val_data,
#         num_epoch=50,
#         monitor=None,
#         eval_metric=LossMetric(0.5),
#         optimizer='rmsprop',
#         optimizer_params={'learning_rate':0.01, 'lr_scheduler': mx.lr_scheduler.FactorScheduler(300000, 0.1, 0.001)},
#         initializer=mx.init.Xavier(magnitude=2, rnd_type='gaussian', factor_type='in'),
#         arg_params=args_params,
#         aux_params=aux_params,
#         allow_missing=True,
#         batch_end_callback=[mx.callback.Speedometer(batch_size=2, frequent=10, auto_reset=False)],
#         epoch_end_callback=checkpoint,
#        )

In [ ]:
# num =5 # the number of batch
# img_num = 0
# train_data.reset()
# for i in range(num):
#     batch = train_data.next()
#     for j in range(len(batch.index)):
#         img_num +=1
#         print(img_num) 
#         img = batch.data[0].asnumpy()[j]
#         label = batch.label[0].asnumpy()[j].reshape(7,7,9)
#         ilist, jlist = np.where(label[:,:,0]==1) 
#         print(ilist,jlist)
# num =5 # the number of batch
# img_num = 0
# val_data.reset()
# for i in range(num):
#     batch = val_data.next()
#     for j in range(len(batch.index)):
#         img_num +=1
#         print(img_num) 
#         img = batch.data[0].asnumpy()[j]
#         label = batch.label[0].asnumpy()[j].reshape(7,7,9)
#         ilist, jlist = np.where(label[:,:,0]==1) 
#         print(ilist,jlist)